### Captum Visual Insights for Finetuned AG News Classification  model from the BERT Pretrained Model 

This Notebook helps you to get started with the Captum Insights. In this example, we have finetuned the BERT pre-trained model using feature extraction for AG News classification which classifies the given input in one of the following classes ("world", "Sports", "Business", "Sci/Tech"). We have showcased that how we can visualize the word importances and attributions.We can visualize Captum Insights normally and making a request on torchserve as well as shown  in this notebook

In [3]:
!mlflow deployments predict --name news_classification_test --target torchserve --input-path input.json --output-path bert_prediction_output.json

In [4]:
! cat bert_prediction_output.json

"\"Business\""

In [25]:
!mlflow deployments explain --name news_classification_test --target torchserve --input-path input.json --output-path bert_explanation_output.json

In [26]:
! cat bert_explanation_output.json

"{\n  \"words\": [\n    \"this\",\n    \"year\",\n    \"business\",\n    \"is\",\n    \"good\"\n  ],\n  \"importances\": [\n    0.026999072020006464,\n    0.4472419670206138,\n    0.607592509685662,\n    -0.6073303341425822,\n    0.24744066046171273\n  ],\n  \"delta\": 0.0017185753788282465\n}"

In [30]:
with open("./bert_explanation_output.json", "r") as fp:
    explanations_json = json.load(fp)
explanations_json = json.loads(explanations_json)

explanations_json


{'words': ['this', 'year', 'business', 'is', 'good'],
 'importances': [0.026999072020006464,
  0.4472419670206138,
  0.607592509685662,
  -0.6073303341425822,
  0.24744066046171273],
 'delta': 0.0017185753788282465}

In [43]:
prediction_json = json.loads(open("./bert_prediction_output.json", "r").read())
prediction_json = json.loads(prediction_json)
prediction_json

'Business'

In [44]:
import torch
attributions = explanations_json['importances']
tokens = explanations_json['words']
delta = explanations_json['delta']

attributions = torch.tensor(attributions)
pred_prob = 0.75
pred_class = prediction_json
true_class = "Business"
attr_class ="world"

In [45]:
from captum.attr import visualization
vis_data_records =[]
vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred_prob,
                            pred_class,
                            true_class,
                            attr_class,
                            attributions.sum(),       
                            tokens,
                            delta))

In [46]:
vis = visualization.visualize_text(vis_data_records)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Business,Business (0.75),world,0.72,this year business is good
